In [1]:
import pandas as pd
import numpy as np
import re, string, nltk

# Data Preprocessing
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split

# Feature Extraction
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Building Model
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


C:\Users\vivek\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\vivek\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [25]:
data_train = pd.read_csv(r'./train.csv')
data_test = pd.read_csv(r'./test.csv')


In [26]:
data_train=data_train.iloc[:5000].copy()

In [27]:
data_test=data_test.iloc[:1000].copy()

In [28]:
data_train.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [29]:
data_train = data_train.fillna('')
data_test = data_test.fillna('')


In [30]:
punc = list(string.punctuation)
stop_word = stopwords.words('english')
lemma = WordNetLemmatizer()
ps = PorterStemmer()


In [31]:
def preprocessing(data):
    data = re.sub('[^a-zA-Z]', ' ', data)
    data = data.lower()
    data = data.split()
    data = ' '.join([ps.stem(word) for word in data if (word not in punc) or (word not in stop_word)])

#     data = ' '.join([lemma.lemmatize(word) for word in data if (word not in punc) or (word not in stop_word)])
    return data


In [32]:
data_train['Final_text'] = data_train['text'].apply(preprocessing)
data_test['Final_text'] = data_test['text'].apply(preprocessing)


In [33]:
data_train.head()


,id,title,author,text,label,Final_text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,hous dem aid we didn t even see comey s letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,ever get the feel your life circl the roundabo...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,whi the truth might get you fire octob the ten...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,video civilian kill in singl us airstrik have ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,print an iranian woman ha been sentenc to six ...


In [34]:
def drop_col(data):
    data = data.drop(columns=['id', 'author', 'title', 'text'], axis=1)
    data = data.dropna()
    return data

data_train = drop_col(data_train)
data_test = drop_col(data_test)


## Feature Extraction

In [35]:
## Bag of Words

In [36]:
bag_of_words = CountVectorizer()
X_bow = bag_of_words.fit_transform(data_train['Final_text']).toarray()
testing_data_bow = bag_of_words.transform(data_test['Final_text']).toarray()
X_bow[:2]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [37]:
Y_bow = data_train['label'].values


In [38]:
## TF-IDF

In [39]:
TF_IDF = TfidfVectorizer()
X_TFIDF = TF_IDF.fit_transform(data_train['Final_text']).toarray()
testing_data_tfidf = TF_IDF.transform(data_test['Final_text']).toarray()


In [40]:
## train-test split

In [42]:
x_train_bow, x_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, Y_bow, train_size=0.8)
x_train_TFIDF, x_test_TFIDF, y_train_TFIDF, y_test_TFIDF = train_test_split(X_TFIDF, Y_bow, train_size=0.8)


## Random Forest

In [43]:
model_bow_rf = RandomForestClassifier()
model_bow_rf.fit(x_train_bow, y_train_bow)

print('Training Score is --> ', model_bow_rf.score(x_train_bow, y_train_bow))
print('Testing Score is --> ', model_bow_rf.score(x_test_bow, y_test_bow))


Training Score is -->  1.0
Testing Score is -->  0.902


In [44]:
model_tfidf_rf = RandomForestClassifier()
model_tfidf_rf.fit(x_train_TFIDF, y_train_TFIDF)

print('Training Score is --> ', model_tfidf_rf.score(x_train_TFIDF, y_train_TFIDF))
print('Testing Score is --> ', model_tfidf_rf.score(x_test_TFIDF, y_test_TFIDF))


Training Score is -->  1.0
Testing Score is -->  0.873
